In [1]:
#import necessary packages
import pandas as pd
import datetime as dt

In [2]:
#load in demographic data and remove "(HUD)" from end of string for race, ethnicity, and veteran status
demo = pd.read_csv("https://raw.githubusercontent.com/datasci611/bios611-projects-fall-2019-arquinter/master/project_3/data/CLIENT_191102.tsv", sep="\t")
for column in demo[["Client Primary Race", "Client Ethnicity", "Client Veteran Status"]]:
    demo[column] = demo[column].astype(str).str.rstrip(" (HUD)")

In [3]:
#load in disability at entry dataset, keep affirmative disability entries for each subject, and remove "(HUD)" from end of
#string for disability determination and type
dis_entry = pd.read_csv("https://raw.githubusercontent.com/datasci611/bios611-projects-fall-2019-arquinter/master/project_3/data/DISABILITY_ENTRY_191102.tsv", sep = "\t")[["Client ID", "Disability Determination (Entry)", "Disability Type (Entry)"]]
dis_entry2 = dis_entry[dis_entry["Disability Determination (Entry)"] == "Yes (HUD)"].drop_duplicates()
for column in dis_entry2[["Disability Determination (Entry)", "Disability Type (Entry)"]]:
    dis_entry2[column] = dis_entry2[column].map(lambda x: x.rstrip(" (HUD)"))

In [4]:
#transpose dataset by disability type so that there is a column for each type of disability
dis_entry3 = pd.pivot(dis_entry2, index = "Client ID", columns = "Disability Type (Entry)", values = "Disability Determination (Entry)").fillna("No")

In [5]:
#add suffix "_entry" to column names and then merge dataset to demographic data#transpose dataset by disability type so that there is a column for each type of disabilityset
dis_entry3.columns = [str(x) + '_Entry' if ("ID" not in x or "AIDS" in x) else x for x in dis_entry3.columns]
dis_entry_fin = demo.merge(dis_entry3, on = "Client ID", how = "left")

In [6]:
#fill in missing values of disability columns with "no"
dis_entry_fin[["Alcohol Abuse_Entry", "Both Alcohol and Drug Abuse_Entry", "Chronic Health Condition_Entry", "Developmental_Entry",
             "Drug Abuse_Entry", "HIV/AIDS_Entry", "Mental Health Problem_Entry", "Other_Entry", "Other: Learning_Entry", "Physical_Entry",
             "Physical/Medical_Entry", "Vision Impaired_Entry"]] = dis_entry_fin[["Alcohol Abuse_Entry", "Both Alcohol and Drug Abuse_Entry",
             "Chronic Health Condition_Entry", "Developmental_Entry", "Drug Abuse_Entry", "HIV/AIDS_Entry", "Mental Health Problem_Entry", "Other_Entry",
             "Other: Learning_Entry", "Physical_Entry", "Physical/Medical_Entry", "Vision Impaired_Entry"]].fillna("No")

In [7]:
#create a new column "disability_entry" that record yes if the client had at least one disability of any type of the 5 of
#interest and fill in missing entries with "no"
for column in dis_entry_fin[["Alcohol Abuse_Entry", "Developmental_Entry",
             "Drug Abuse_Entry", "Mental Health Problem_Entry", "Other_Entry"]]:
    dis_entry_fin.loc[(dis_entry_fin[column] == "Yes"), 'Disability_Entry'] = "Yes"
dis_entry_fin['Disability_Entry'] = dis_entry_fin['Disability_Entry'].fillna("No")

In [8]:
#load in disability at exit dataset, keep affirmative disability entries for each subject, and remove "(HUD)" from end of
#string for disability determination and type
dis_exit = pd.read_csv("https://raw.githubusercontent.com/datasci611/bios611-projects-fall-2019-arquinter/master/project_3/data/DISABILITY_EXIT_191102.tsv", sep = "\t")[["Client ID", "Disability Determination (Exit)", "Disability Type (Exit)"]]
dis_exit2 = dis_exit[dis_exit["Disability Determination (Exit)"] == "Yes (HUD)"].drop_duplicates()
for column in dis_exit2[["Disability Determination (Exit)", "Disability Type (Exit)"]]:
    dis_exit2[column] = dis_exit2[column].map(lambda x: x.rstrip(" (HUD)"))

In [9]:
#transpose dataset by disability type so that there is a column for each type of disability
dis_exit3 = pd.pivot(dis_exit2, index = "Client ID", columns = "Disability Type (Exit)", values = "Disability Determination (Exit)").fillna("No")

In [10]:
#add suffix "_exit" to column names and then merge dataset to demographic dataset
dis_exit3.columns = [str(x) + '_Exit' if ("ID" not in x or "AIDS" in x) else x for x in dis_exit3.columns]

In [11]:
dis_exit_fin = demo.merge(dis_exit3, on = "Client ID", how = "left")

In [12]:
dis_exit_fin[["Alcohol Abuse_Exit", "Both Alcohol and Drug Abuse_Exit", "Chronic Health Condition_Exit", "Developmental_Exit",
             "Drug Abuse_Exit", "HIV/AIDS_Exit", "Mental Health Problem_Exit", "Other_Exit", "Other: Learning_Exit", "Physical_Exit",
             "Physical/Medical_Exit", "Vision Impaired_Exit"]] = dis_exit_fin[["Alcohol Abuse_Exit", "Both Alcohol and Drug Abuse_Exit",
             "Chronic Health Condition_Exit", "Developmental_Exit", "Drug Abuse_Exit", "HIV/AIDS_Exit", "Mental Health Problem_Exit", "Other_Exit",
             "Other: Learning_Exit", "Physical_Exit", "Physical/Medical_Exit", "Vision Impaired_Exit"]].fillna("No")

In [13]:
dis_exit_fin.drop(dis_exit_fin.columns[[4, 5, 6, 7, 8, 9]], axis = 1, inplace = True)

In [14]:
#create a new column "disability_exit that record yes if the client had at least one disability of any type of the 5 of
#interest and fill in missing entries with "no"
for column in dis_exit_fin[["Alcohol Abuse_Exit", "Developmental_Exit",
             "Drug Abuse_Exit", "Mental Health Problem_Exit", "Other_Exit"]]:
    dis_exit_fin.loc[(dis_exit_fin[column] == "Yes"), 'Disability_Exit'] = "Yes"
dis_exit_fin['Disability_Exit'] = dis_exit_fin['Disability_Exit'].fillna("No")

In [15]:
#merge disability at entry and disability at exit datasets
disability_data = dis_entry_fin.merge(dis_exit_fin, on = ["Client ID", "EE Provider ID", "EE UID", "Client Unique ID"], how = "inner")

In [16]:
#fill in blanks with "missing" and replace missing key "nan" that was entered by hand with "missing"
disability_data["Client Gender"] = disability_data["Client Gender"].fillna("Missing")
disability_data["Client Primary Race"] = disability_data["Client Primary Race"].replace({"nan": "Missing"})

In [17]:
#create new variable that records yes if a client had at least one of the 5 disabilities of interest at entry and no disability
#upon their exit
disability_data.loc[((disability_data["Disability_Exit"] == "No")&(disability_data["Disability_Entry"] == "Yes")),
                 'Disability_Change'] = "Yes"
disability_data["Disability_Change"] = disability_data["Disability_Change"].fillna("No")

In [18]:
#load in entry/exit dataset and drop observations with missing exit date, as duration variable requires that both entry and
#exit dates are known
ee_dates = pd.read_csv("https://raw.githubusercontent.com/datasci611/bios611-projects-fall-2019-arquinter/master/project_3/data/ENTRY_EXIT_191102.tsv", sep="\t")
ee_dates = ee_dates.dropna(subset=["Exit Date"])

In [19]:
#create datetime versions of entry and exit date variables
ee_dates["entry_num"] = ee_dates["Entry Date"].map(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
ee_dates["exit_num"] = ee_dates["Exit Date"].map(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))

In [20]:
#create length of stay variable and merge to demographics dataset
ee_dates["dur"] = ee_dates["exit_num"] - ee_dates["entry_num"]
duration_data = ee_dates.merge(demo, on = ['Client ID', 'EE Provider ID', 'EE UID', 'Client Unique ID'], how = 'left')

In [21]:
#drop extra variable that was created during merge
duration_data = duration_data.drop("Unnamed: 6", axis = 1)

In [22]:
#extract days part of datetime length of stay variable as numeric variable
duration_data["dur"] = duration_data["dur"].dt.days

In [23]:
#merge duration dataset to disability dataset and export to csv
analysis_data = disability_data.merge(duration_data[['Client ID', 'EE Provider ID', 'EE UID', 'Client Unique ID', 'dur']],
                             on = ['Client ID', 'EE Provider ID', 'EE UID', 'Client Unique ID'], how = 'left')
analysis_data.to_csv("./analysis_data.csv")